In [ ]:
#import packages

In [ ]:
from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import time 
# import keyboard
import time
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
# from simple_pid import PID
import os

In [ ]:
#connecting to devices 
rm = pyvisa.ResourceManager()
#rm = pyvisa.ResourceManager()

In [ ]:
#ASRL7 = Linear Stage
com_pxpy = rm.open_resource('ASRL7::INSTR')
com_pxpy.baud_rate = 921600 #fixed

com_pxpy.write('1MO')
com_pxpy.write('2MO')

#syringe pump
com_syr = rm.open_resource('ASRL4::INSTR')
# com_syr.baud_rate = 115200

In [ ]:
#Reset acceleration, velocities, position

#acceleration
com_pxpy.write('1HA'+str(1))
com_pxpy.write('2HA'+str(1))

#group velocity
com_pxpy.write('1HV'+str(1))
com_pxpy.write('2HV'+str(1))

#velocity #actually impacts experiments
com_pxpy.write('1VA'+str(10))
com_pxpy.write('2VA'+str(10))

#position
com_pxpy.write('1PA'+str(0))
com_pxpy.write('2PA'+str(0))

In [ ]:
#check position
live_position_x_t = com_pxpy.query_ascii_values('1TP?')
live_position_x = ''.join(str(element) for element in live_position_x_t)
live_position_y_t = com_pxpy.query_ascii_values('2TP?')
live_position_y = ''.join(str(element) for element in live_position_y_t)
print(live_position_x)
print(live_position_y)

In [ ]:
#Defining an operation that executes the pumping procedure
def single_pump(sy_dia, res_vol, ID, rate, dispense_vol):
    #sy_dia = 12.03 
    #ID = 0, 1, 2 (pump number)
    #res_vol = reservoir volume in ml
    #rate = pump rate in ul/min
    #sample_vol = volume to be pumped at defined rate

    com_syr.write(str(ID) + 'svolume ' + str(res_vol) + ' ml')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'diameter ' + str(float(sy_dia)))
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irate ' + str(float(rate)) + ' ul/min')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'tvolume ' + str(float(dispense_vol)) + ' ul')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irun')

In [ ]:
#Initializing deposition process
space = s = 10 #spacing of deposition steps

position_x = [0*s,1*s,2*s,
              0*s,1*s,2*s]
position_y = [0*s,0*s,0*s,
              1*s,1*s,1*s]

In [ ]:
#fixed parameters
sy_dia = 12.03

#changed parameters
res_vol = 0.4 #mL #same for 1,2 

rate_0 = Q = 10 #uL/min
#rate_1 = [0.1*Q, Q, 10*Q]
rate_1 = Q

dispense_vol_0 = 10
dispense_vol_1 = 10
    #dispense_vol_1 = np.empty(len(rate_1))
    #for i in range(len(rate_1)):
    #    dispense_vol_1[i] = dispense_vol_0*(rate_1[i]/rate_0)

t = (dispense_vol_0/Q) * 60

In [ ]:
#"unlooped"
single_pump(sy_dia, res_vol, 0, rate_0, dispense_vol_0)
single_pump(sy_dia, res_vol, 1, rate_1, dispense_vol_1)
time.sleep(t)

In [ ]:
for i in range(len(rate_1)): 
    #com_pxpy.write('1PA'+str(position_x[i]))
    #com_pxpy.write('2PA'+str(position_y[i]))
    #single_pump(sy_dia, res_vol, 0, rate_0, dispense_vol_0)
    #single_pump(sy_dia, res_vol, 1, rate_1[i], dispense_vol_1[i])
    print(position_x[i],position_y[i])
    time.sleep(2)
    
    #add camera
    